In [6]:
# default_exp core

# graph_utils 

> a library for working with graphs

In [7]:
#hide
from nbdev.showdoc import *

# declare base `Graph` class

In [31]:
# export
import abc
import numpy as np

class Graph(abc.ABC):
    
    def __init__(self, numVertices:int, directed:bool=False):
        self.numVertices = numVertices
        self.directed = directed
        
    def check_validity(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Vertice {} is out of bounds".format(v))
        return True
    
    @abc.abstractmethod
    def add_edge(self, v1, v2, weight):
        pass
    
    @abc.abstractmethod
    def get_adjacent_vertices(self, v):
        pass
    
    @abc.abstractmethod
    def get_indegree(self, v):
        pass
    
    @abc.abstractmethod
    def get_edge_weight(self, v1, v2):
        pass
    
    @abc.abstractmethod
    def display(self):
        pass
        

# AdjacencyMatrixGraph

In [32]:
# export

class AdjacencyMatrixGraph(Graph):
    def __init__(self, numVertices:int, directed:bool=False):
        super(AdjacencyMatrixGraph, self).__init__(numVertices, directed)
        
        self.matrix = np.zeros([numVertices, numVertices])
        

    def add_edge(self, v1, v2, weight=1):

        self.check_validity(v1)
        self.check_validity(v2)
        if weight < 1:
            raise ValueError("An edge cant have weight < 1")
        
        self.matrix[v1][v2] = weight
            
    def get_adjacent_vertices(self, v):
        self.check_validity(v)

        adjacent_vertices = []

        for i in range(self.numVertices):
            if self.matrix[v][i] > 0:
                adjacent_vertices.append(i)
        return adjacent_vertices

    def get_edge_weight(self, v1, v2):
        self.check_validity(v1)
        self.check_validity(v2)
        return self.matrix[v1][v2]

    def get_indegree(self, v):
        self.check_validity(v)

        indegree = 0
        for i in range(self.numVertices):
            if self.matrix[v][i] > 0:
                indegree += 1
        return indegree

    def display(self):
        print(str(self.matrix))

In [33]:
###################################################
# test AdjacencyMatrixGraph 
####################################################

amg = AdjacencyMatrixGraph(numVertices=3)
amg.add_edge(1, 2)
amg.add_edge(0, 1)
amg.add_edge(0, 2)

got = amg.matrix.tolist()
expected = [[0.0, 1.0, 1.0],
            [0.0, 0.0, 1.0],
            [0.0, 0.0, 0.0]]
assert got == expected, "got: {}, expected: {}".format(got, expected)

amg.add_edge(1, 1, weight=5)
got = amg.matrix.tolist()
expected = [[0.0, 1.0, 1.0],
            [0.0, 5.0, 1.0],
            [0.0, 0.0, 0.0]]
assert got == expected, "got: {}, expected: {}".format(got, expected)


assert amg.get_indegree(1) == 2
assert amg.get_edge_weight(1,1) == 5
assert amg.get_adjacent_vertices(1) == [1,2]

# AdjacencySetGraph

each node maintainst a set of it's adjascend nodes

![](https://i.imgur.com/KsMWFni.png)

In [46]:
# export
class Node:
    """Node represents one vertex in a graph
    
    each node has a vertex id
    each node is associated with a set of adjacent vertices
    """
    def __init__(self, vertexId):
        self.vertexId = vertexId
        self.adjacency_set = set()
        
    def add_edge(self, v):
        if self.vertexId == v:
            raise ValueError("Vertex {} cannot be adjacent to itself".format(v))
        else:
            self.adjacency_set.add(v)
            
    def get_adjacent_vertices(self):
        return sorted(self.adjacency_set)
    
    def __repr__(self):
        return "Node({})".format(self.vertexId)

In [56]:
class AdjacencySetGraph(Graph):
    
    def __init__(self, numVertices, directed=False):
        super(AdjacencySetGraph, self).__init__(numVertices, directed)
        self.vertex_list = [Node(i) for i in range(numVertices)]

    def add_edge(self, v1, v2, weight=1):
        self.check_validity(v1)
        self.check_validity(v2)
        if weight != 1:
            raise ValueError("An adjacency set can only represent edge weights == 1")
            
        # add connection from v1 to v2
        self.vertex_list[v1].add_edge(v2)
        
        # undirected graphs work both ways
        if self.directed == False:
            self.vertex_list[v2].add_edge(v1)

    def get_adjacent_vertices(self, v):
        self.check_validity(v)
        
        node = self.vertex_list[v]
        return node.get_adjacent_vertices()
    
    def get_indegree(self, v):
        self.check_validity(v)
        
        indegree = 0
        
        for i in range(self.numVertices):
            if v in self.get_adjacent_vertices(i):
                indegree += 1
                
        return indegree
    
    def get_edge_weight(self, v1, v2):
        return 1
        
    
    def display(self):
        for node in asg.vertex_list:
            for v in node.get_adjacent_vertices():
                print(node.vertexId, "-->", v)
        
    
asg = AdjacencySetGraph(4, directed=True)
asg.vertex_list

[Node(0), Node(1), Node(2), Node(3)]

In [60]:
asg.add_edge(0, 1)
asg.add_edge(0, 3)
asg.add_edge(1, 2)
asg.add_edge(2, 3)
asg.add_edge(3, 0)
asg.display()

0 --> 1
0 --> 3
1 --> 2
2 --> 3
3 --> 0


# Breadth-first graph traversal

> scan all nodes 1 step away from origin, then 2 steps, then 3, and so on

In [64]:
1+2

3

# Depth-first graph traversal

> go deep in 1 direction until you reach a dead end

In [63]:
1+1

2